## Import the required libraries

In [36]:
import openrouteservice
import folium
from itertools import permutations
from IPython.display import display

## Initialization

In [39]:
client = openrouteservice.Client(key = "5b3ce3597851110001cf6248940c956ef1b941d7aa32623ece4bfe7c")

## Route Fetching Function

In [42]:
def get_route_between_points(start, end):
    route = client.directions(
        coordinates = [(start[1], start[0]), (end[1], end[0])],
        profile = "driving-car",
        format = "geojson"
    )
    return route

## Brute Force Route Calculation

In [34]:
def calculate_route_brute_force(locations, distance_matrix):
    from itertools import permutations
    num_locations = len(locations)
    shortest_distance = float("inf")
    best_route = None

    for perm in permutations(range(1, num_locations)):
        total_distance = distance_matrix[0][perm[0]]
        for i in range(1, len(perm)):
            total_distance += distance_matrix[perm[i-1]][perm[i]]
        total_distance += distance_matrix[perm[-1]][0]

    if total_distance < shortest_distance:
        shortest_distance = total_distance
        best_route = perm

    return best_route, shortest_distance

## Fetching Distance Matrix

In [44]:
def get_distance_matrix(locations):
    coords = [[loc[1], loc[0]] for loc in locations]
    matrix = client.distance_matrix(coords, metrics = ["distance"])
    return matrix["distances"]

## Plotting Route on Map

In [58]:
def plot_route_on_map(locations, best_route):
    m = folium.Map(location = locations[0], zoom_start = 13)

    folium.Marker(locations[0], popup = "Start/End", icon = folium.Icon(color = "green")).add_to(m)
    for i, loc in enumerate(best_route, start = 1):
        folium.Marker(locations[loc], popup = f"Location {i}").add_to(m)

    route_coords = [locations[0]] + [locations[i] for i in best_route] + [locations[0]]

    for i in range(len(route_coords) - 1):
        route = get_route_between_points(route_coords[i], route_coords[i + 1])
        geometry = route["features"][0]["geometry"]["coordinates"]
        folium.PolyLine([(coord[1], coord[0]) for coord in geometry], color = "blue", weight = 2.5, opacity = 1).add_to(m)

    display(m)

## Main Function

In [61]:
def main():
    user_name = input("Enter your name here: ")
    n = int(input("Enter the number of locations to visit (including start/end point) here: "))
    locations = []
    for i in range(n):
        lat = float(input(f"Enter the latitude for location {i+1}: "))
        lon = float(input(f"Enter the longitude for location {i+1}: "))
        locations.append((lat, lon))

    print("Fetching distance matrix from OpenRouteService...")
    distance_matrix = get_distance_matrix(locations)

    best_route, shortest_distance = calculate_route_brute_force(locations, distance_matrix)

    print(f"\nHello {user_name}, the optimal route for visiting your locations is:")
    print(locations[0], "->", end="")
    for i in best_route:
        print(locations[i], "->", end="")
    print(locations[0])

    print(f"Total distance: {shortest_distance / 1000:.2f} km")

    plot_route_on_map(locations, best_route)

if __name__ == "__main__":
    main()

Enter your name here:  KS
Enter the number of locations to visit (including start/end point) here:  2
Enter the latitude for location 1:  34.058126497020844
Enter the longitude for location 1:  -117.82142947952937
Enter the latitude for location 2:  34.0599766733906
Enter the longitude for location 2:  -117.82425561815788


Fetching distance matrix from OpenRouteService...

Hello KS, the optimal route for visiting your locations is:
(34.058126497020844, -117.82142947952937) ->(34.0599766733906, -117.82425561815788) ->(34.058126497020844, -117.82142947952937)
Total distance: 2.52 km
